In [1]:
!pip install interpret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 45.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 60.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 66.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 62.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.8/778.8 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.8/247.8 kB 11.1 MB/s eta 0:00:00
  Created wheel for dash-cytoscape: filename=dash_cytoscape-1.0.1-py3-none-any.whl size=4010600 sha256=d888a79c6bfa7b0e36e94a45bca7a256c63b05c573163bbcf9510b4ea060e316
  Stored in directory: /root/.cache/pip/wheels/2b/86/e6/a00e9295bb80ac2d86b93b9cbf5ba1ae78e9f739d75398e41f
Successfully built dash-cytoscape


In [2]:
import os
if os.environ.get('PWD') == '/kaggle/working':
    os.chdir('/kaggle/input/cusser-data')

In [3]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show

In [4]:
def createGlassbox(filename,excludeCols=['path']):
    df = pd.read_csv(filename,index_col=False)
    df.reset_index(drop=True, inplace=True)
    df = df.drop(columns=excludeCols)

    unique_subjects = df['ActorID'].unique()
    # Step 2: Shuffle and split subjects into train, test, and validation sets
    train_subjects, temp_subjects = train_test_split(unique_subjects, test_size=0.3, random_state=42)

    # Step 3: Create DataFrames for each split based on the subject IDs
    train_df = df[df['ActorID'].isin(train_subjects)]
    test_df = df[df['ActorID'].isin(temp_subjects)]


    y_train = train_df["emotion"]
    y_test = test_df["emotion"]


    x_train=train_df.drop(columns=["emotion","ActorID"])
    x_test=test_df.drop(columns=["emotion","ActorID"])


    ebm = ExplainableBoostingClassifier()
    ebm.fit(x_train, y_train)

    auc = roc_auc_score(y_test, ebm.predict_proba(x_test),multi_class='ovr')
    print(("AUC: {:.3f}".format(auc)))
    
    show(ebm.explain_global())
    return ebm


In [ ]:
createGlassbox("data/opensmileDF_functional.csv",  excludeCols = ["path","Unnamed: 0"])

In [ ]:
createGlassbox("data/crema_avd.csv")

In [ ]:
#Combining Opensmile and AVD features

df1 = pd.read_csv("data/crema_avd.csv")
df2 = pd.read_csv("data/opensmileDF_functional.csv")

df3 = df1.merge(df2,how="left", on=["ActorID","emotion","path"])
df3=df3.drop(columns="Unnamed: 0")
df3.head()
df3.to_csv("data/crema_opensmile_avd.csv",index=False)

In [ ]:
toExclude = ["path","Age","Sex","Race","Ethnicity"]
createGlassbox("data/crema_opensmile_avd.csv",excludeCols =toExclude)